# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 48 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.03963


extracting tarball to tmp_2209.03963...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.03967


extracting tarball to tmp_2209.03967...

...

/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.04343/newWXCha.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure ColCol_diag_XW.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/tmp/ipykernel_2254/4030337529.py:34: LatexWarning: 2209.04343 did not run properly
Could not find figure ColCol_diag_XW.png
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2209.04369


extracting tarball to tmp_2209.04369...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.04304-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.04304) | **Accretion and extinction variations in the low-mass pre-main sequence  binary system WX Cha**  |
|| Eleonora Fiorellino, et al. -- incl., <mark>Agnes Kospal</mark> |
|*Appeared on*| *2022-09-12*|
|*Comments*| **|
|**Abstract**| Light curves of young star systems show photometric variability due to different kinematic, and physical processes. One of the main contributors to the photometric variability is the changing mass accretion rate, which regulates the interplay between the forming young star and the protoplanetary disk. We collected high-resolution spectroscopy in eight different epochs, as well as ground-based and space-borne multi-epoch optical and infrared photometry of WX Cha, an M0 binary system, with an almost edge-on disk (i = 87degrees) in the Chamaeleon I star-forming region. Spectroscopic observations cover 72 days, the ground-based optical monitoring covers 42 days while space-borne TESS photometry extends for 56 days. The multi-wavelength light curves exhibit quasi-periodic variability of 0.35 - 0.53 mag in the near-infrared, and of 1.3 mag in g band. We studied the variability of selected emission lines that trace the accretion, computed the accretion luminosity and the mass accretion rate using empirical relations and obtained values of the accretion luminosity between 1.6 and 3.2 Lsun and mass accretion rate between 3.31x10{-7} Msun/yr and 7.76x10^{-7} Msun/yr. Our results show that WX Cha is accreting at a rate larger than what is typical for T Tauri stars in the same star-forming region with the same stellar parameters. We theorize that this is due to the higher disk mass of WX Cha than what is usual for stars with similar stellar mass, and to the binary nature of the system. Daily changes in the accretion luminosity and in the extinction can explain the photometric variability. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03972-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03972) | **COOL-LAMPS III: Discovery of a 25".9 Separation Quasar Lensed by a  Merging Galaxy Cluster**  |
|| Michael N. Martinez, et al. -- incl., <mark>Guillaume Mahler</mark> |
|*Appeared on*| *2022-09-12*|
|*Comments*| *13 pages, 6 figures. Submitted to ApJ*|
|**Abstract**| In the third paper from the COOL-LAMPS Collaboration, we report the discovery of COOL J0542-2125, a gravitationally lensed quasar at $z=1.84$, observed as three images due to an intervening massive galaxy cluster at $z=0.61$. The lensed quasar images were identified in a search for lens systems in recent public optical imaging data and have separations on the sky up to 25".9, wider than any previously known lensed quasar. The galaxy cluster acting as a strong lens appears to be in the process of merging, with two sub-clusters separated by $\sim 1$ Mpc in the plane of the sky, and their central galaxies showing a radial velocity difference of $\sim 1000$ km/s. Both cluster cores show strongly lensed images of an assortment of background sources, as does the region between them. A preliminary strong lens model implies masses of $M(<250\ \rm{kpc}) = 1.79^{+0.16} _{-0.01} \times 10^{14} M_{\odot}$ and $M(<250\ \rm{kpc}) = 1.48^{+0.04}_{-0.10} \times 10^{14} M_{\odot}$ for the East and West sub-clusters, respectively. This line of sight is also coincident with a ROSAT ALL-sky Survey source, centered between the two confirmed cluster halos reminiscent of other major cluster-scale mergers. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.04322-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.04322) | **The MegaMapper: A Stage-5 Spectroscopic Instrument Concept for the Study  of Inflation and Dark Energy**  |
|| David J. Schlegel, et al. -- incl., <mark>Joseph DeRose</mark>, <mark>Daniel Green</mark>, <mark>Henry Heetderks</mark>, <mark>Dustin Lang</mark> |
|*Appeared on*| *2022-09-12*|
|*Comments*| *Contributed White Paper to Snowmass 2021. arXiv admin note: substantial text overlap with arXiv:1907.11171. text overlap with arXiv:2209.03585*|
|**Abstract**| In this white paper, we present the MegaMapper concept. The MegaMapper is a proposed ground-based experiment to measure Inflation parameters and Dark Energy from galaxy redshifts at $2<z<5$. In order to achieve path-breaking results with a mid-scale investment, the MegaMapper combines existing technologies for critical path elements and pushes innovative development in other design areas. To this aim, we envision a 6.5-m Magellan-like telescope, with a newly designed wide field, coupled with DESI spectrographs, and small-pitch robots to achieve multiplexing of at least 26,000. This will match the expected achievable target density in the redshift range of interest and provide a 10x capability over the existing state-of the art, without a 10x increase in project budget. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.04282-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.04282) | **Nano-grain depletion in photon-dominated regions**  |
|| <mark>T. Schirmer</mark>, et al. |
|*Appeared on*| *2022-09-12*|
|*Comments*| **|
|**Abstract**| Context. Carbonaceous nano-grains play a fundamental role in the physico-chemistry of the interstellar medium (ISM) and especially of photon-dominated regions (PDRs). Their properties vary with the local physical conditions and affect the local chemistry and dynamics. Aims. We aim to highlight the evolution of carbonaceous nano-grains in three different PDRs and propose a scenario of dust evolution as a response to the physical conditions. Methods. We used Spitzer/IRAC (3.6, 4.5, 5.8, and 8 $\mu$m) and Spitzer/MIPS (24 $\mu$m) together with Herschel/PACS (70 $\mu$m) to map dust emission in IC63 and the Orion Bar. To assess the dust properties, we modelled the dust emission in these regions using the radiative transfer code SOC together with the THEMIS dust model. Results. Regardless of the PDR, we find that nano-grains are depleted and that their minimum size is larger than in the diffuse ISM (DISM), which suggests that the mechanisms that lead nano-grains to be photo-destroyed are very efficient below a given critical size limit. The evolution of the nano-grain dust-to-gas mass ratio with both G0 and the effective temperature of the illuminating star indicates a competition between the nano-grain formation through the fragmentation of larger grains and nano-grain photo-destruction. We modelled dust collisions driven by radiative pressure with a classical 1D approach to show that this is a viable scenario for explaining nano-grain formation through fragmentation and, thus, the variations observed in nano-grain dust-to-gas mass ratios from one PDR to another. Conclusions. We find a broad variation in the nano-grain dust properties from one PDR to another, along with a general trend of nano-grain depletion in these regions. We propose a viable scenario of nano-grain formation through fragmentation of large grains due to radiative pressure-induced collisions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.04310-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.04310) | **Constraining $νΛ$CDM with density-split clustering**  |
|| Enrique Paillas, et al. -- incl., <mark>Arnaud de Mattia</mark> |
|*Appeared on*| *2022-09-12*|
|*Comments*| *Submitted to MNRAS. Source code for all figures in the paper is provided in the captions*|
|**Abstract**| The dependence of galaxy clustering on local density provides an effective method for extracting non-Gaussian information from galaxy surveys. The two-point correlation function (2PCF) provides a complete statistical description of a Gaussian density field. However, the late-time density field becomes non-Gaussian due to non-linear gravitational evolution and higher-order summary statistics are required to capture all of its cosmological information. Using a Fisher formalism based on halo catalogues from the Quijote simulations, we explore the possibility of retrieving this information using the density-split clustering (DS) method, which combines clustering statistics from regions of different environmental density. We show that DS provides more precise constraints on the parameters of the $\nu \Lambda$CDM model compared to the 2PCF, and we provide suggestions for where the extra information may come from. DS improves the constraints on the sum of neutrino masses by a factor of $8$ and by factors of 5, 3, 4, 6, and 6 for $\Omega_m$, $\Omega_b$, $h$, $n_s$, and $\sigma_8$, respectively. We compare DS statistics when the local density environment is estimated from the real or redshift-space positions of haloes. The inclusion of DS autocorrelation functions, in addition to the cross-correlation functions between DS environments and haloes, recovers most of the information that is lost when using the redshift-space halo positions to estimate the environment. We discuss the possibility of constructing simulation-based methods to model DS clustering statistics in different scenarios. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.04369-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.04369) | **Higgsless simulations of cosmological phase transitions and  gravitational waves**  |
|| Ryusuke Jinno, Thomas Konstandin, <mark>Henrique Rubira</mark>, Isak Stomberg |
|*Appeared on*| *2022-09-12*|
|*Comments*| *30 pages, 10 figures*|
|**Abstract**| First-order cosmological phase transitions in the early Universe source sound waves and, subsequently, a background of stochastic gravitational waves. Currently, predictions of these gravitational waves rely heavily on simulations of a Higgs field coupled to the plasma of the early Universe, the former providing the latent heat of the phase transition. Numerically, this is a rather demanding task since several length scales enter the dynamics. From smallest to largest, these are the thickness of the Higgs interface separating the different phases, the shell thickness of the sound waves, and the average bubble size. In this work, we present an approach to perform Higgsless simulations in three dimensions, producing fully nonlinear results, while at the same time removing the hierarchically smallest scale from the lattice. This significantly reduces the complexity of the problem and contributes to making our approach highly efficient. We provide spectra for the produced gravitational waves for various choices of wall velocity and strength of the phase transition, as well as introduce a fitting function for the spectral shape. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.04343-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.04343) | **The relation between the Mass Accretion Rate and the Disk Mass in Class  I Protostars**  |
|| Eleonora Fiorellino, et al. -- incl., <mark>Agnes Kospal</mark> |
|*Appeared on*| *2022-09-12*|
|*Comments*| **|
|**Abstract**| The evidence of a relation between the mass accretion rate and the disk mass is established for young, Class II pre-main sequence stars. This observational result opened an avenue to test theoretical models and constrain the initial conditions of the disk formation, fundamental in the understanding of the emergence of planetary systems. However, it is becoming clear that the planet formation starts even before the Class II stage, in disks around Class 0 and I protostars. We show for the first time evidence for a correlation between the mass accretion rate and the disk mass for a large sample of Class I young stars located in nearby (< 500 pc) star-forming regions. We fit our sample, finding that the Class I objects relation has a slope flatter than Class II stars, and have higher mass accretion rates and disk masses. The results are put in context of the disk evolution models. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure ColCol_diag_XW.png</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.04165-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.04165) | **Open questions in massive star research**  |
|| <mark>N. Langer</mark> |
|*Appeared on*| *2022-09-12*|
|*Comments*| *to appear in Proceedings for IAU Symposium 361: Massive Stars Near and Far, held in Ballyconnell, Ireland, 9-13 May 2022, N. St-Louis, J. S. Vink & J. Mackey, eds*|
|**Abstract**| In discussing open question in the field of massive stars, I consider their evolution from birth to death. After touching upon massive star formation, which may be bi-modal and not lead to a zero-age main sequence at the highest masses, I consider the consequences of massive stars being close to their Eddington limit. Then, when discussing the effects of a binary companion, I highlight the importance of massive Algols and contact binaries for understanding the consequences of mass transfer, and the role of binaries in forming Wolf-Rayet stars. Finally, a discussion on pair instability supernovae and of superluminous supernovae is provided. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.04304.md
    + _build/html/tmp_2209.04304/./veiling_time_mine_Av_mean_peter.png
    + _build/html/tmp_2209.04304/./Lacc_time_Av_mean_peter.png
    + _build/html/tmp_2209.04304/./Lacc_Lstar_comparison_Av_mean_peter.png
    + _build/html/tmp_2209.04304/./wxcha_lc_period.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\avWX}{2.75\pm0.75}$
$\newcommand{\lsun}{\mbox{L}_\odot}$
$\newcommand{\rsun}{\mbox{R}_\odot}$
$\newcommand{\msun}{\mbox{M}_\odot}$
$\newcommand{\lacc}{L_{\rm acc}}$
$\newcommand{\macc}{\dot{M}_{\rm acc}}$
$\newcommand{\lstar}{L_\star}$
$\newcommand{\mstar}{M_\star}$
$\newcommand{\rstar}{R_\star}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\lbol}{L_{\rm bol}}$
$\newcommand{\zsg}[1]{\textcolor{green}{[GZs: #1]}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\avWX$}{2.75\pm0.75}$
$\newcommand{$\lsun$}{\mbox{L}_\odot}$
$\newcommand{$\rsun$}{\mbox{R}_\odot}$
$\newcommand{$\msun$}{\mbox{M}_\odot}$
$\newcommand{$\lacc$}{L_{\rm acc}}$
$\newcommand{$\macc$}{\dot{M}_{\rm acc}}$
$\newcommand{$\lstar$}{L_\star}$
$\newcommand{$\mstar$}{M_\star}$
$\newcommand{$\rstar$}{R_\star}$
$\newcommand{$\teff$}{T_{\rm eff}}$
$\newcommand{$\lbol$}{L_{\rm bol}}$
$\newcommand{$\zsg$}[1]{\textcolor{green}{[GZs: #1]}}$</div>



<div id="title">

# Based on observations collected at the European Southern Observatory under ESO programmes 2103.C-5025, 0103.A-9008, and 0100.C-0708(A).

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.04304-b31b1b.svg)](https://arxiv.org/abs/2209.04304)<mark>Appeared on: 2022-09-12</mark> - __

</div>
<div id="authors">

Eleonora Fiorellino, et al.

</div>
<div id="abstract">

**Abstract:** Light curves of young star systems show photometric variability due to different kinematic, and physical processes.  One of the main contributors to the photometric variability is the changing mass accretion rate, which regulates the interplay between the forming young star and the protoplanetary disk.  We collected high-resolution spectroscopy in eight different epochs, as well as ground-based and space-borne multi-epoch optical and infrared photometry of WX Cha, an M0 binary system, with an almost edge-on disk ($i =87^\circ$) in the Chamaeleon I star-forming region.  Spectroscopic observations cover 72 days, the ground-based optical monitoring covers 42 days while space-borne TESS photometry extends for 56 days.  The multi-wavelength light curves exhibit quasi-periodic variability of$0.35-0.53$mag in the near-infrared, and of 1.3 mag in$g$band. We studied the variability of selected emission lines that trace the accretion, computed the accretion luminosity and the mass accretion rate using empirical relations and obtained values between$\lacc \sim 1.6   $\lsun$   -   3.2   $\lsun$$and$\macc \sim 3.31 \times 10^{-7}  $\msun$/{\rm yr}   -   7.76 \times 10^{-7}$$\msun/$yr.  Our results show that WX Cha is accreting at a rate larger than what is typical for T Tauri stars in the same star-forming region with the same stellar parameters. We theorize that this is due to the higher disk mass of WX Cha than what is usual for stars with similar stellar mass, and to the binary nature of the system.  Daily changes in the accretion luminosity and in the extinction can explain the photometric variability.

</div>

<div id="div_fig1">

<img src="tmp_2209.04304/./veiling_time_mine_Av_mean_peter.png" alt="Fig3.1" width="50%"/><img src="tmp_2209.04304/./Lacc_time_Av_mean_peter.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** {\it Top}: Absolute veiling measurements for WX Cha.
    {\it Bottom}: The accretion luminosity and the mass accretion rate as a function of the time. The filled circles represent accretion rates computed by using the mean value ($A_V = 3.4 \pm 0.2$). The filled squares represent the accretion rates computed by using the exact value of the extinction for that epoch (see Tab. \ref{tab:Av}). (*fig:veiling*)

</div>
<div id="div_fig2">

<img src="tmp_2209.04304/./Lacc_Lstar_comparison_Av_mean_peter.png" alt="Fig4" width="100%"/>

**Figure 4. -** {\it Top}: Accretion luminosity as a function of the stellar luminosity. {\it Bottom}: Mass accretion rate as a function of the stellar mass. Black stars show Cha I CTTS sample \citep[][]{manaraPPVII}. In particular, the star with error bar corresponds to WX Cha.
    Yellow, purple, blue and dark green circles correspond to CR Cha, VW Cha, DR Tau, and RW Tau accretion rates \citep[][respectively]{zsidi2022, zsidi2022vwcha, giannini2022, facchini2016}. WX Cha accretion rates computed in this work are shown in hexagons, each epoch is identified by the a different color as labelled in the legend. The black cross on the bottom-right of the plot represent the uncertainty for Cha I CTTS sources. The uncertainties smaller than the symbol size are not presented. Dashed-dotted lines and the grey regions represent the best fit of Cha I sources and the relative $1\sigma$ dispersion. \label{fig:acc_stellarpar} (*fig:acc_stellarpar*)

</div>
<div id="div_fig3">

<img src="tmp_2209.04304/./wxcha_lc_period.png" alt="Fig6" width="100%"/>

**Figure 6. -** The 2019 (top panel) TESS light curve is indicated with black dots, and the contemporaneous ASAS SN $g$ band light curve is shown with green points. SMARTS $I$-band (red), $J$(pink), $H$(bordeaux), and $K$(brown) light curves are also added.
    Vertical dashed lines correspond to epochs when ESPRESSO (orange) and FEROS (magenta) spectra were taken. Since Epoch 2 (FEROS) and 3 (ESPRESSO) were taken during the same night, so these epochs are superimposed in the figure.
    The blue line (bottom panel) shows the Lomb-Scargle periodogram obtained using the 2019 TESS data. Points of the same light curve are linked with dashed lines only to help the reader follow each light curve. Horizontal lines represent false alarm probabilities of 1\%, and 0.01\%, respectively. (*fig:lc_period*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

30  publications in the last 7 days.
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers